<a href="https://colab.research.google.com/github/haji8-de/AIFFEL_quest_rs/blob/main/Exploration/Ex03/ablation_study_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchinfo

In [ ]:
# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torchinfo import summary
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Q. 이미지의 표현이 0과 1 사이로 들어오도록 직접 Pytorch 정규화 코드를 작성해봅시다.
transform = transforms.Compose([
    transforms.ToTensor()  # uint8 → float32 변환 + 0~1 정규화
    # 이미지 픽셀 값을 255로 나누어 0과 1 사이로 정규화하기 위해 transforms.ToTensor()를 사용한다.
])

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    return np.transpose(npimg, (1, 2, 0))


In [ ]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder


hyperparams = {
    "batch_size": 4,
    "learning_rate": 0.0001,
    "epochs": 5,
    "transform": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48235, 0.45882, 0.40784],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    ),
}

In [ ]:
trainset = torchvision.datasets.OxfordIIITPet(root='./data', split='trainval', download=True, target_types=["binary-category","category"], transform=hyperparams['transform'])
testset = torchvision.datasets.OxfordIIITPet(root='./data', split='test', download=True, target_types=["binary-category","category"], transform=hyperparams['transform'])

trainloader_2 = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader_2 = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# trainloader_2 = torch.utils.data.DataLoader(ds_split['train'], batch_size=32, shuffle=True)
# testloader_2 = torch.utils.data.DataLoader(ds_split['test'], batch_size=32, shuffle=False)
# validationloader_2 = torch.utils.data.DataLoader(ds_split['validation'], batch_size=32, shuffle=False)

In [ ]:
def show_multiple_images_binary_label(dataset, n_images=9):
    dataiter = iter(dataset)
    images, labels = next(dataiter)
    fig, axes = plt.subplots(3, 3, figsize=(6, 6))
    axes = axes.flatten()

    # OxfordIIITPet 데이터셋의 이진 레이블에 대한 이름을 정의합니다.
    # 'binary-category'는 일반적으로 0:cat, 1:dog을 나타냅니다.
    binary_labels_map = ['cat', 'dog']

    for i in range(n_images):
        ax = axes[i]
        img = imshow(images[i])
        ax.imshow(img)

        # labels는 (binary_labels_batch_tensor, category_labels_batch_tensor) 형태이므로
        # 첫 번째 텐서(binary_labels_batch_tensor)에서 i번째 항목을 가져옵니다.
        binary_label_idx = labels[0][i].item()

        # 인덱스가 유효한 범위 내에 있는지 확인하고, 아니면 'Unknown'으로 처리합니다.
        if 0 <= binary_label_idx < len(binary_labels_map):
            label_text = binary_labels_map[binary_label_idx]
        else:
            label_text = f"Unknown (Index: {binary_label_idx})"

        ax.set_title(f"Label: {label_text}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# trainloader_2 에 대한 이진 레이블 시각화
print("Trainloader_2 (Binary Labels):")
show_multiple_images_binary_label(trainloader_2)

# testloader_2 에 대한 이진 레이블 시각화
print("\nTestloader_2 (Binary Labels):")
show_multiple_images_binary_label(testloader_2)

In [ ]:
from torch import nn


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes,
            kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    inplanes, self.expansion*planes,
                    kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [ ]:
class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes,
            kernel_size=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(
            planes, self.expansion*planes,
            kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    inplanes, self.expansion*planes,
                    kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [ ]:
import torch


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super().__init__()

        self.inplanes = 64
        self.stem = nn.Sequential(
            nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.inplanes),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.stage1 = self._make_layer(block, 64, layers[0], stride=1)
        self.stage2 = self._make_layer(block, 128, layers[1], stride=2)
        self.stage3 = self._make_layer(block, 256, layers[2], stride=2)
        self.stage4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        layers = []
        layers.append(block(self.inplanes, planes, stride))
        self.inplanes = planes * block.expansion
        for _ in range(num_blocks - 1):
            layers.append(block(self.inplanes, planes, 1))

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.stem(x)
        out = self.stage1(out)
        out = self.stage2(out)
        out = self.stage3(out)
        out = self.stage4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [ ]:
from torchvision import models
from torchinfo import summary


resnet18 = ResNet(BasicBlock, [2, 2, 2, 2], 1000)
resnet34 = ResNet(BasicBlock, [3, 4, 6, 3], 1000)
resnet50 = ResNet(BottleneckBlock, [3, 4, 6, 3], 1000)
resnet101 = ResNet(BottleneckBlock, [3, 4, 23, 3], 1000)
resnet152 = ResNet(BottleneckBlock, [3, 8, 36, 3], 1000)
torch_model = models.resnet34(weights="ResNet34_Weights.IMAGENET1K_V1")

resnet34_info = summary(resnet34, (1, 3, 224, 224), verbose=0)
torch_model_info = summary(torch_model, (1, 3, 224, 224), verbose=0)

print(resnet34_info.total_params)
print(torch_model_info.total_params)

In [ ]:

class build_resnet():
  def __init__(self, is_50, input_shape=(32,32,3), num_classes=1000):
    self._is_50 = is_50 # Store the parameter as an instance attribute
    self._input_shape = input_shape # Store input_shape for potential future use
    self._num_classes = num_classes # Store num_classes

    if self._is_50: # Use the instance attribute to determine the model type
      self.model = ResNet(BottleneckBlock, [3, 4, 6, 3], self._num_classes)
      self.block_type = BottleneckBlock
    else:
      self.model = ResNet(BasicBlock, [3, 4, 6, 3], self._num_classes)
      self.block_type = BasicBlock

  def forward(self, x):
    return self.model(x)

  def summary(self):
    # Use standard input size for ResNet summary, e.g., for ImageNet
    return summary(self.model, (1, 3, 224, 224), verbose=0)

In [ ]:
# @title ResNet34 Summary
resnet_34_2 = build_resnet(is_50=False, input_shape=(32, 32,3), num_classes=2)
print(resnet_34_2.summary())

In [ ]:
# @title ResNet50 Summary
resnet_50_2 = build_resnet(is_50=True, input_shape=(32, 32,3), num_classes=2)
print(resnet_50_2.summary())

In [ ]:
# @title model 설정 : ResNet50
model = resnet50

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=hyperparams["learning_rate"])

In [ ]:

for images, classes in trainloader_2:
    images = images.to(device)
    classes = classes[0][1]#.item()
    print(classes)

In [ ]:
import torch
import gc

# Free up memory
torch.cuda.empty_cache()
gc.collect()

In [ ]:
for epoch in range(hyperparams["epochs"]):
    cost = 0.0

    for images, classes in trainloader_2:
        images = images.to(device)
        
        # --- THE FIX ---
        # classes is a list: [binary_targets, category_targets]
        # 1. Select the second tensor (Index 1 is "category")
        target_labels = classes[1] 
        
        # 2. Move ONLY that tensor to the device
        target_labels = target_labels.to(device)
        # ---------------

        output = model(images)
        
        # Ensure target is the correct shape/type for loss (usually Long)
        # If your output is shape [32, 10], target_labels should be [32]
        loss = criterion(output, target_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss.item() # .item() converts it to a simple float, freeing the GPU memory

    cost = cost / len(trainloader_2)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}, Loss : {loss.item():.3f}")

In [ ]:

with torch.no_grad():
    model.eval()

    accuracy = 0.0
    for images, classes in testloader_2:
        images = images.to(device)
        classes = classes[1].to(device)

        outputs = model(images)
        probs = F.softmax(outputs, dim=-1)
        outputs_classes = torch.argmax(probs, dim=-1)

        accuracy += int(torch.eq(classes, outputs_classes).sum())

    print(f"acc@1 : {accuracy / (len(testloader) * hyperparams['batch_size']) * 100:.2f}%")

In [ ]:
def show_multiple_images_binary_label(dataset, n_images=9):
    dataiter = iter(dataset)
    images, labels = next(dataiter)
    fig, axes = plt.subplots(3, 3, figsize=(6, 6))
    axes = axes.flatten()

    # OxfordIIITPet 데이터셋의 이진 레이블에 대한 이름을 정의합니다.

    for i in range(n_images):
        ax = axes[i]
        img = imshow(images[i])
        ax.imshow(img)

        # labels는 (binary_labels_batch_tensor, category_labels_batch_tensor) 형태이므로
        # 첫 번째 텐서(binary_labels_batch_tensor)에서 i번째 항목을 가져옵니다.


        # 인덱스가 유효한 범위 내에 있는지 확인하고, 아니면 'Unknown'으로 처리합니다.
        if 0 <= binary_label_idx < len(binary_labels_map):
            label_text = binary_labels_map[binary_label_idx]
        else:
            label_text = f"Unknown (Index: {binary_label_idx})"

        ax.set_title(f"Label: {label_text}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# 'binary-category'는 일반적으로 0:cat, 1:dog을 나타냅니다.
binary_labels_map = ['cat', 'dog']
def display_predictions_binary_label(loader, model, class_names, device, n_images=9):
    dataiter = iter(loader)
    images, labels = next(dataiter) # Get a batch of images and labels

    fig, axes = plt.subplots(3, 3, figsize=(8, 8)) # Create a 3x3 grid for images
    axes = axes.flatten() # Flatten the 2D array of axes for easier iteration

    with torch.no_grad(): # Disable gradient calculation for inference
        for i in range(n_images):
            if i >= len(images): # Ensure we don't go out of bounds if batch size is smaller than n_images
                break

            ax = axes[i]
            img = imshow(images[i]) # Use the helper function to properly display the image
            ax.imshow(img)

            # Get model output for the image
            outputs = model(images[i].unsqueeze(0).to(device)) # Add batch dimension and move to device

            # Get predicted class
            _, predicted = torch.max(outputs.data, 1)

            # Get the class names
            predicted_class_name = class_names[predicted.item()]
            print(predicted_class_name)
            
            binary_label_idx = labels[0][i].item()
            # true_class_name = class_names[labels[i]]

            # Set the title with true and predicted labels
            # 인덱스가 유효한 범위 내에 있는지 확인하고, 아니면 'Unknown'으로 처리합니다.
            if 0 <= binary_label_idx < len(binary_labels_map):
                label_text = binary_labels_map[binary_label_idx]
            else:
                label_text = f"Unknown (Index: {binary_label_idx})"

            ax.set_title(f"True: {binary_label_idx}, Label: {predicted_class_name}")
            # ax.set_title(f"True: {true_class_name}\nPred: {predicted_class_name}", fontsize=10)
            ax.axis('off') # Hide axes ticks

    plt.tight_layout() # Adjust subplot parameters for a tight layout
    plt.show() # Display the plot


In [ ]:
class_names = binary_labels_map
display_predictions_binary_label(trainloader_2, model, class_names, device, n_images=9)

In [ ]:
class_names = ['cat', 'dog']

In [ ]:
for epoch in range(hyperparams["epochs"]):
    cost = 0.0

    for images, classes in trainloader:
        images = images.to(device)
        classes = classes.to(device)

        output = model(images)
        loss = criterion(output, classes)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(trainloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")
    # print('Epoch %s: Train Accuracy: %.2f percent, Validation Accuracy: %.2f percent, Train Loss: %s, Validation Loss: %s'
    #       % (epoch, train_acc, val_acc, train_loss, val_loss))

In [ ]:

with torch.no_grad():
    model.eval()

    accuracy = 0.0
    for images, classes in testloader:
        images = images.to(device)
        classes = classes.to(device)

        outputs = model(images)
        probs = F.softmax(outputs, dim=-1)
        outputs_classes = torch.argmax(probs, dim=-1)

        accuracy += int(torch.eq(classes, outputs_classes).sum())

    print(f"acc@1 : {accuracy / (len(testloader) * hyperparams['batch_size']) * 100:.2f}%")

In [ ]:
def display_predictions(loader, model, class_names, device, n_images=9):
    dataiter = iter(loader)
    images, labels = next(dataiter) # Get a batch of images and labels

    fig, axes = plt.subplots(3, 3, figsize=(8, 8)) # Create a 3x3 grid for images
    axes = axes.flatten() # Flatten the 2D array of axes for easier iteration

    with torch.no_grad(): # Disable gradient calculation for inference
        for i in range(n_images):
            if i >= len(images): # Ensure we don't go out of bounds if batch size is smaller than n_images
                break

            ax = axes[i]
            img = imshow(images[i]) # Use the helper function to properly display the image
            ax.imshow(img)

            # Get model output for the image
            outputs = model(images[i].unsqueeze(0).to(device)) # Add batch dimension and move to device

            # Get predicted class
            _, predicted = torch.max(outputs.data, 1)

            # Get the class names
            predicted_class_name = class_names[predicted.item()]
            true_class_name = class_names[labels[i]]

            # Set the title with true and predicted labels
            ax.set_title(f"True: {true_class_name}\nPred: {predicted_class_name}", fontsize=10)
            ax.axis('off') # Hide axes ticks

    plt.tight_layout() # Adjust subplot parameters for a tight layout
    plt.show() # Display the plot


In [ ]:

display_predictions(trainloader, model, class_names, device)
display_predictions(testloader, model, class_names, device)

In [ ]:

build_plainnet(is_50=False)

build_plainnet(is_50=True)
